In [281]:
import pandas as pd

In [282]:
df = pd.read_excel(r"processed_df.xlsx",engine="openpyxl")

In [283]:
df.head(2)

,city,transmission,ownerNo,oem,model,modelYear,variantName,price,Registration Year,Fuel Type,Kms Driven,Engine Displacement,Transmission,Mileage
0,Bangalore,Manual,3,Maruti,Maruti Celerio,2015,VXI,₹ 4 Lakh,2015,Petrol,"1,20,000 Kms",998 cc,Manual,23.1 kmpl
1,Bangalore,Manual,2,Ford,Ford Ecosport,2018,1.5 Petrol Titanium BSIV,₹ 8.11 Lakh,Feb 2018,Petrol,"32,706 Kms",1497 cc,Manual,17 kmpl


In [284]:
# Rename_columns
df.rename(columns={"ownerNo": "owner_no",
                   "oem": "brand",
                   "modelYear": "model_year",
                   "variantName": "variant_name",
                   "Registration Year": "registered_year",
                   "Fuel Type": "fuel_type",
                   "Kms Driven": "kms_driven",
                   "Engine Displacement": "engine_cc",
                   "Transmission": "transmission",
                   "Year of Manufacture": "manufacture_year",
                   "Mileage": "mileage_kmpl"},inplace=True)

# Strip_spaces_in_column_names
df.columns = df.columns.str.strip()

In [285]:
# Strip all white/trailing spaces & if string convert it to lowercase for consistency
df = df.apply(lambda col: col.str.strip().str.lower() if col.dtype == 'object' else col,axis=0)

In [286]:
df.head(2)

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,kms_driven,engine_cc,transmission,mileage_kmpl
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,₹ 4 lakh,2015,petrol,"1,20,000 kms",998 cc,manual,23.1 kmpl
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,₹ 8.11 lakh,feb 2018,petrol,"32,706 kms",1497 cc,manual,17 kmpl


In [287]:
# remove 'kms' and make it numerical standard
df["kms_driven"] = df["kms_driven"].replace({"kms": "",",": ""},regex=True).str.strip()

# engine displacement column remove unwanted strings
df["engine_cc"] = df["engine_cc"].str.replace("cc","").str.strip()

# remove "kmph" in mileage_kmph column
df["mileage_kmpl"] = df["mileage_kmpl"].str.replace("kmpl","").str.strip()

# remove unwanted strings and keep only year
df["registered_year"] = df["registered_year"].str.extract(r'\b(\d{4})\b').astype(float) #float will handle Nan value

In [288]:
#drop duplicate records
df.drop_duplicates(inplace=True)

In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8253 entries, 0 to 8368
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             8253 non-null   object 
 1   transmission     8253 non-null   object 
 2   owner_no         8253 non-null   int64  
 3   brand            8253 non-null   object 
 4   model            8253 non-null   object 
 5   model_year       8253 non-null   int64  
 6   variant_name     8253 non-null   object 
 7   price            8253 non-null   object 
 8   registered_year  8201 non-null   float64
 9   fuel_type        8253 non-null   object 
 10  kms_driven       8251 non-null   object 
 11  engine_cc        8249 non-null   object 
 12  transmission     8253 non-null   object 
 13  mileage_kmpl     7970 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 967.1+ KB


In [290]:
# drop rows where registered_year is null (else it will skew data)
df = df.dropna(subset=['registered_year'])

In [291]:
df[(df["kms_driven"].isnull()) | (df["engine_cc"].isnull())] # these values needs to be filled

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,kms_driven,engine_cc,transmission,mileage_kmpl
574,bangalore,automatic,1,mahindra,mahindra e2o plus,2017,p6,₹ 5.50 lakh,2017.0,electric,20000,NaN,automatic,110
5510,hyderabad,manual,1,mercedes-benz,mercedes-benz c-class,2002,180 elegance,₹ 2.25 lakh,2002.0,petrol,135000,NaN,manual,NaN
5683,hyderabad,manual,3,tata,tata indigo,2007,lx,"₹ 20,161",2007.0,diesel,NaN,1405,manual,17
6430,jaipur,manual,1,maruti,maruti wagon r,2015,vxi bs iv,₹ 3.50 lakh,2015.0,petrol,NaN,998,manual,20.51


In [292]:
# convert to numeric datatypes
df["engine_cc"] = pd.to_numeric(df["engine_cc"],errors="coerce")
df["kms_driven"] = pd.to_numeric(df["kms_driven"],errors="coerce")
df['mileage_kmpl'] = pd.to_numeric(df['mileage_kmpl'], errors='coerce')

# fill engine_cc using grouping and mean
df["engine_cc"] = df.groupby(['brand','model','model_year','fuel_type'])['engine_cc'].transform(lambda x: x.fillna(x.mean()))

# fill kms_driven using grouping and mean
df["kms_driven"] = df.groupby(['brand','model','model_year','fuel_type'])['kms_driven'].transform(lambda x: x.fillna(x.mean()))

# fill mileage_kmpl using grouping and mean
df['mileage_kmpl'] = df.groupby(['brand','engine_cc','fuel_type'])['mileage_kmpl'].transform(lambda x: x.fillna(x.mean()))

In [293]:
#drop null value rows for specific columns
df.dropna(subset=['engine_cc','kms_driven'],inplace=True)

In [294]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8199 entries, 0 to 8368
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             8199 non-null   object 
 1   transmission     8199 non-null   object 
 2   owner_no         8199 non-null   int64  
 3   brand            8199 non-null   object 
 4   model            8199 non-null   object 
 5   model_year       8199 non-null   int64  
 6   variant_name     8199 non-null   object 
 7   price            8199 non-null   object 
 8   registered_year  8199 non-null   float64
 9   fuel_type        8199 non-null   object 
 10  kms_driven       8199 non-null   float64
 11  engine_cc        8199 non-null   float64
 12  transmission     8199 non-null   object 
 13  mileage_kmpl     8132 non-null   float64
dtypes: float64(4), int64(2), object(8)
memory usage: 960.8+ KB


In [295]:
#drop unfilled column using mean and grouping
df.dropna(subset=['mileage_kmpl'],inplace=True)

In [296]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8132 entries, 0 to 8368
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             8132 non-null   object 
 1   transmission     8132 non-null   object 
 2   owner_no         8132 non-null   int64  
 3   brand            8132 non-null   object 
 4   model            8132 non-null   object 
 5   model_year       8132 non-null   int64  
 6   variant_name     8132 non-null   object 
 7   price            8132 non-null   object 
 8   registered_year  8132 non-null   float64
 9   fuel_type        8132 non-null   object 
 10  kms_driven       8132 non-null   float64
 11  engine_cc        8132 non-null   float64
 12  transmission     8132 non-null   object 
 13  mileage_kmpl     8132 non-null   float64
dtypes: float64(4), int64(2), object(8)
memory usage: 953.0+ KB
